## Requirements

### Web-Scraping
#### 1. Take the base wg-gesucht URL and scrape the shared flats listed on the website
#### 2. most important data points are price, room size and number of flat mates

### Data Analyis
#### 1. Correlation analysis of room price and room size. Hypothesis: positive correlation
#### 2. Correlation analysis of room price and number of flat mates. Hypothesis: negative correlation

### Import all neccessary libraries

In [4]:
import requests

### Set base wg-gesucht url
### Add string interpolation to allow easy iteration through listing pages

In [5]:
pageNumber = 0
URL = f'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.{pageNumber}.html?pagination=1&pu='


### Iterate through listing pages and scrape the required data points
### Place scraped data into a dataframe/dictionary

In [7]:
for i in range(0,5): 
    requests.get(URL)
    pageNumber += 1
    